In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import seaborn as sns
#from statsmodels.graphics.mosaicplot import mosaic
import io
import requests

%matplotlib inline


import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import log_loss

pd.set_option('display.max_columns', None)

In [57]:
df4 = pd.read_csv('dataset_v4.csv').set_index('building_id')
train_labels = pd.read_csv('train_labels.csv').set_index('building_id')

In [62]:
# df4 es mi set de entrenamiento
train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [7]:
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier

In [63]:
x, y = df4, train_labels

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2048)

In [69]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208480 entries, 285985 to 370619
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   geo_level_1_id                          208480 non-null  int64
 1   geo_level_2_id                          208480 non-null  int64
 2   geo_level_3_id                          208480 non-null  int64
 3   age                                     208480 non-null  int64
 4   area_percentage                         208480 non-null  int64
 5   height_percentage                       208480 non-null  int64
 6   has_superstructure_adobe_mud            208480 non-null  bool 
 7   has_superstructure_mud_mortar_stone     208480 non-null  bool 
 8   has_superstructure_stone_flag           208480 non-null  bool 
 9   has_superstructure_cement_mortar_stone  208480 non-null  bool 
 10  has_superstructure_mud_mortar_brick     208480 non-null  bool 


# RandomForest


In [70]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
clfRF = RandomForestClassifier(n_estimators=50, max_features="log2")

In [ ]:
clfRF.fit(x_train, y_train)

/home/fscaccheri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [ ]:
y_pred = clfRF.predict(x_test)
accuracy_score(y_test, y_pred)

# XGBoost

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [ ]:
arr_x = np.array(x)
arr_y = np.array(y)

In [ ]:
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(arr_y, arr_y):
    xgb_model = xgb.XGBRFClassifier(random_state=40).fit(
    arr_x[train_index], arr_y[train_index])

In [ ]:
score = xgb_model.score(x, y)
print("accuracy=%.2f%%" % ( score * 100))

# VotingClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
eclf = VotingClassifier(estimators=[('rf', clfRF),('xgb', xgb_model)], voting='hard')

In [ ]:
eclf = eclf.fit(x_train, y_train)

In [ ]:
y_pred =eclf.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# Ahora entrenamos con el set completo de datos
eclf = eclf.fit(x, y)

In [ ]:
test_values = pd.read_csv('encoded_test_values.csv').set_index('building_id')

In [ ]:
test_values = test_values[
    ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
     'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered',
       'roof_type_C0', 'roof_type_C1', 'land_surface_condition_C1',
       'land_surface_condition_C0', 'foundation_type_C0', 'foundation_type_C1',
       'foundation_type_C2', 'ground_floor_type_C0', 'ground_floor_type_C1',
       'ground_floor_type_C2', 'other_floor_type_C0', 'other_floor_type_C1',
       'other_floor_type_C2', 'position_C0', 'position_C1', 'position_C2',
       'plan_configuration_C0', 'plan_configuration_C1',
       'plan_configuration_C2', 'plan_configuration_C3'
    ]
]

for col in test_values.columns:
    if col.startswith('has_superstructure'):
        test_values[col] = test_values[col].astype(bool)

test_values.info()

In [ ]:
prediction =eclf.predict(test_values)

In [ ]:
sub_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
submission = pd.DataFrame(data=prediction, columns=sub_format.columns, index=sub_format.index)
submission.to_csv('submission_ensamble.csv')